In [1]:
import random

In [2]:
population_size = 150
chromosome_length = 30
generations = 69
mutation_rate = 0.1
tournament_size = 10

In [3]:
def create_population(population_size, chromosome_length):
    population = []
    for _ in range(population_size):
        individual = [random.randint(0, 1) for _ in range(chromosome_length)]
        population.append(individual)
    return population


def fitness_function(individual):
    return chromosome_length-sum(individual)

In [4]:
def selection_tournament(population, fitness_values, tournament_size=tournament_size):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(range(len(population)), tournament_size)
        winner = tournament[0]
        for competitor in tournament[1:]:
            if fitness_values[competitor] > fitness_values[winner]:
                winner = competitor
        selected.append(population[winner])
    return selected


def selection_roulette(population, fitness_values):
    total_fitness = sum(fitness_values)
    selection_probabilities = [fitness / total_fitness for fitness in fitness_values]
    selected = []
    for _ in range(len(population)):
        r = random.random()
        cumulative_prob = 0
        for i, prob in enumerate(selection_probabilities):
            cumulative_prob += prob
            if r <= cumulative_prob:
                selected.append(population[i])
                break
    return selected

In [5]:
def crossover_single_point(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2


def crossover_two_point(parent1, parent2):
    crossover_point1 = random.randint(1, len(parent1) - 2)
    crossover_point2 = random.randint(crossover_point1 + 1, len(parent1) - 1)
    child1 = parent1[:crossover_point1] + parent2[crossover_point1:crossover_point2] + parent1[crossover_point2:]
    child2 = parent2[:crossover_point1] + parent1[crossover_point1:crossover_point2] + parent2[crossover_point2:]
    return child1, child2

In [6]:
def mutate_flip(individual, mutation_rate):
    mutated_individual = individual[:]
    for i in range(len(mutated_individual)):
        if random.random() < mutation_rate:
            mutated_individual[i] = 1 - mutated_individual[i]
    return mutated_individual


def mutate_inversion(individual, mutation_rate):
    mutated_individual = individual[:]
    if random.random() < mutation_rate:
        mutation_point1 = random.randint(0, len(mutated_individual) - 1)
        mutation_point2 = random.randint(0, len(mutated_individual) - 1)
        start = min(mutation_point1, mutation_point2)
        end = max(mutation_point1, mutation_point2)
        mutated_individual[start:end+1] = reversed(mutated_individual[start:end+1])
    return mutated_individual


def mutate_swap(individual, mutation_rate):
    mutated_individual = individual[:]
    if random.random() < mutation_rate:
        mutation_point1 = random.randint(0, len(mutated_individual) - 1)
        mutation_point2 = random.randint(0, len(mutated_individual) - 1)
        mutated_individual[mutation_point1], mutated_individual[mutation_point2] = \
            mutated_individual[mutation_point2], mutated_individual[mutation_point1]
    return mutated_individual

In [7]:
def genetic_algorithm(population_size, chromosome_length, generations,
                      mutation_rate, selection_method, crossover_method,
                      mutation_method):
    population = create_population(population_size, chromosome_length)
    for _ in range(generations):
        fitness_values = [fitness_function(individual) for individual in population]

        selected = selection_method(population, fitness_values)

        offspring = []
        for i in range(0, population_size, 2):
            parent1 = selected[i]
            parent2 = selected[i + 1]

            child1, child2 = crossover_method(parent1, parent2)

            offspring.append(mutation_method(child1, mutation_rate))
            offspring.append(mutation_method(child2, mutation_rate))

        population = offspring

    best_individual = max(population, key=fitness_function)
    return best_individual, fitness_function(best_individual)

In [8]:
selection_method = selection_roulette
crossover_method = crossover_two_point
mutation_method = mutate_flip

best_solution, best_fitness = genetic_algorithm(population_size, chromosome_length,
                                                generations, mutation_rate, selection_method,
                                                crossover_method, mutation_method)

In [9]:
print("Best solution:", best_solution)
print("Best fitness:", best_fitness)

Best solution: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
Best fitness: 23
